In [ ]:
%load_ext autoreload
%autoreload 2

from setup_imports import *  # noqa: F401,F403
from src.phrases.utils import generate_phrase_hash
from src.wiktionary.lookup import get_wiktionary_urls
import json
from src.phrases.search import (
    get_phrases_by_collection,
    get_verbs_from_phrases,
    get_vocab_from_phrases,
)
from src.story import get_story
from src.challenges.models import ChallengeRecord, get_challenge
from langcodes import Language

In [ ]:
story = get_story(story_title_hash="a_winter_morning_walk_d7b5b3")

In [ ]:
target_lang = Language.get("sv-SE")
print(target_lang.display_name())

In [ ]:
story.publish_story(target_language=target_lang)

In [ ]:
challenge = get_challenge(story_title_hash="a_winter_morning_walk_d7b5b3")

In [ ]:
challenge.publish(target_language=target_lang, overwrite=True)

In [ ]:
story = get_story(story_title_hash="flying_to_suprise_the_family_9bb91e")

In [ ]:
story.publish_story(target_language="sv-SE", overwrite=True)

In [ ]:
challenge = ChallengeRecord.create(story)

In [ ]:
challenge.upload()

In [ ]:
challenge.publish(target_language="sv-SE", overwrite=True)

In [ ]:
challenge.model_dump()

In [ ]:
Challenge.model_validate(challenge)

In [ ]:
"""Test script to render a challenge HTML from your example challenge data."""


# Your example challenge data
challenge_data = {
    "scenarios": [
        {
            "role_learner": "person going for a walk",
            "role_teacher": "walking companion",
            "situation": "Two friends preparing to go for a winter walk outside their home",
            "difficulty": "easy",
            "task": "Get ready for the walk and find out what to bring",
            "find_out": [
                {
                    "question": "What clothing items should be worn?",
                    "answer": "Blue jeans and a hat",
                },
                {"question": "What drink should be brought along?", "answer": "Coffee"},
                {
                    "question": "What should be done before leaving?",
                    "answer": "Check how cold it is by opening the window",
                },
            ],
        },
        {
            "role_learner": "lost walker in the park",
            "role_teacher": "walking companion",
            "situation": "Two people walking in a snowy park who realize they might be lost",
            "difficulty": "medium",
            "task": "Recognize you're lost and figure out how to get help",
            "find_out": [
                {
                    "question": "Does your companion think you're going the wrong way?",
                    "answer": "Yes, they don't recognize this part of the park",
                },
                {
                    "question": "What solution does your companion suggest?",
                    "answer": "Ask someone for directions",
                },
                {
                    "question": "Where should you wait while they ask?",
                    "answer": "Wait here while they talk to the person",
                },
            ],
        },
        {
            "role_learner": "person receiving directions",
            "role_teacher": "walking companion who just got directions",
            "situation": "In a park after getting lost, your companion has just returned with directions from a helpful stranger",
            "difficulty": "hard",
            "task": "Find out the directions and confirm you understand the way home",
            "find_out": [
                {
                    "question": "What are the directions to get back?",
                    "answer": "Go straight and then turn right at the large store",
                },
                {
                    "question": "Why do you recognize the location now?",
                    "answer": "The store is memorable and means we're close to home",
                },
                {
                    "question": "What would you like to do before going home?",
                    "answer": "Drink the coffee together because it's cold",
                },
            ],
        },
    ]
}

# Template context
context = {
    "title": "A Winter Morning Walk",
    "target_language": "Swedish",
    "target_language_code": "sv-SE",
    "source_language": "British English",
    "source_language_code": "en-GB",
    "scenarios": challenge_data["scenarios"],
}

from src.utils import render_html_content


html_content = render_html_content(context, "challenge_template.html")
with open("../outputs/test.html", "w", encoding="utf-8") as f:
    f.write(html_content)

In [ ]:
response.scenarios[2].find_out

In [ ]:
COLLECTION = "LM1000"
DECK = "Pack01"

phrases = get_phrases_by_collection(COLLECTION, DECK)

In [ ]:
p = phrases[0]
get_wiktionary_urls(p.translations["sr-RS"].tokens, "sr")

In [ ]:
len(get_vocab_from_phrases(phrases))

In [ ]:
get_verbs_from_phrases(phrases)

In [ ]:
text = "Measure the pH level"

In [ ]:
tokens = text.split(" ")

In [ ]:
_get_wiktionary_url

In [ ]:
get_wiktionary_url("pH", "pH", "en")

In [ ]:
with open(
    "../outputs/wiktionary_dump/wiktionary_dump.json", "r", encoding="utf-8"
) as f:
    for step, line in enumerate(f):
        data = json.loads(line)
        print(data)
        break

In [ ]:
"Sam! Har du sett det här offentliga meddelandet om allemansrätten? Det öppnas en ny vandringsled imorgon."

In [ ]:
story_title = "A Test Story"
story_summary = "A brief summary of the test story."
story_title_hash = generate_phrase_hash(story_title)
story_dialogue = load_json("../data/test_story.json")

story_title2 = "A Test Story 2"
story_summary2 = "A brief summary of the test story 2."
story_title_hash2 = generate_phrase_hash(story_title2)
story_dialogue2 = load_json("../data/test_story2.json")

In [ ]:
ts1 = Story.create(
    story_title, story_summary, story_dialogue, collection="Test", deck="Pack01"
)
ts2 = Story.create(
    story_title2, story_summary2, story_dialogue2, collection="Test2", deck="Pack01"
)
ts1.upload(overwrite=True)
ts2.upload(overwrite=True)